`nn.Transformer`モジュールは、Attensionメカニズム（最近、
[nn.MultiheadAttention](https://pytorch.org/docs/master/nn.html?highlight=multiheadattention#torch.nn.MultiheadAttention)として実装された別のモジュール）に完全に依存しています。このAttensionメカニズムにより、入力と出力の間の大域的な依存関係を捉えることができます。

`nn.Transformer`モジュールは、単一でも使用しやすいように、高度にモジュール化されており、簡単に、`nn.Transformer`モジュールをコンポーネント（本チュートリアルの[nn.TransformerEncoder](https://pytorch.org/docs/master/nn.html?highlight=nn%20transformerencoder#torch.nn.TransformerEncoder)など）に適用したり、あるいは`nn.Transformer`モジュールを使って、コンポーネントを作成したりすることができます。


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
!pip3 install torchtext==0.4.0

  Using cached torchtext-0.4.0-py3-none-any.whl (53 kB)


In [3]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/kawai/anaconda3/envs/pppm/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [7]:
class TransformerModel(nn.Module):
    def __init__(self,ntoken,ninp,nhead,nhid,nlayers,dropout=0.5):
        super(TransformerModel,self).__init__()
        from torch.nn import TransformerEncoder,TransformerEncoderLayer
        self.model_type = "transformer"
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp,dropout)
        encoder_layers = TransformerEncoderLayer(ninp,nhead,nhid,dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers,nlayers)
        self.encoder = nn.Embedding(ntoken,ninp)
        self.ninp = ninp
        self.decoder = nn.Linear(ninp,ntoken)

        self.init_weigths()

    def _generate_square_subsequent_mask(self,sz):
        mask = (torch.triu(torch.ones(sz,sz)) == 1).transpose(0,1)
        mask = mask.float().masked_fill(mask == 0,float('-inf')).masked_fill(mask == 1, float(0,0))
        return mask
    
    def init_weight(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange,initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange,initrange)
    
    def forword(self,src):
        if self.src_mask is None or self.src_mask.size(0) != src.size(0):
            device = src.device
            mask = self._generate_square_subsequent_mask(src.size(0)).to(device)
            self.src_mask = mask
        
        src = self.encoder(src)*math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src,self.src_mask)
        output = self.decoder(output)
        return output
